##### Copyright 2019 Les auteurs de TensorFlow.

In [ ]:
#@title Sous licence Apache, Version 2.0 (la "Licence");
# vous ne pouvez pas utiliser ce fichier, sauf conformément avec la licence.
# Vous pouvez obtenir une copie de la Licence sur
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Sauf si requis par la loi en vigueur ou par accord écrit, le logiciel
# distribué sous la licence est distribué "TEL QUEL", SANS GARANTIE NI
# CONDITION DE QUELQUE NATURE QUE CE SOIT, implicite ou explicite.
# Consultez la Licence pour connaître la terminologie spécifique régissant
# les autorisations et les limites prévues par la Licence.

# TensorFlow 2 démarrage rapide pour avancés

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/advanced"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Voir sur TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/fr/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Exécuter dans Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/fr/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Voir la source sur GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/fr/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Télécharger le notebook</a>
  </td>
</table>

Ceci est un notebook [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb). Les programmes Python sont directement exécutés dans le navigateur - une excellente façon d'apprendre et d'utiliser TensorFlow. Pour suivre ce tutoriel, lancez le notebook dans Google Colab en cliquant sur le bouton en haut de cette page.

1. Dans Colab, connectez-vous à un runtime Python : En haut à droite de la barre de menu, sélectionnez *CONNECTER*.
2. Exécutez toutes les cellules de code : Sélectionnez *Exécution* > *Tout exécuter*.

Téléchargez et installez le paquet TensorFlow 2:

In [ ]:
try:
  # %tensorflow_version existe uniquement dans Colab.
  %tensorflow_version 2.x
except Exception:
  pass


Importez TensorFlow dans votre programme:

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Chargez et préparez le [jeu de données MNIST](http://yann.lecun.com/exdb/mnist/).

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Ajoute une dimension de canaux
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

Utilisez `tf.data` pour regrouper et mélanger l'ensemble de données:

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Construisez le modèle `tf.keras` en utilisant l'[API de sous-classement de modèle](https://www.tensorflow.org/guide/keras#model_subclassing) provenant de Keras:

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Crée une instance du modèle
model = MyModel()

Choisissez une fonction d'optimisation et de perte pour l'entraînement:

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

Sélectionnez les métriques pour mesurer la perte et la précision du modèle. Ces métriques accumulent les valeurs des epochs et écrivent ensuite le résultat global.

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Utilisez `tf.GradientTape` pour entraîner le modèle:

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Testez le modèle:

In [ ]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Réinitialiser les métriques pour le prochain epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Le classificateur d'images est maintenant entraîné avec une précision d'environ 98 % sur cet ensemble de données. Pour en savoir plus, lisez les [Tutoriels TensorFlow](https://www.tensorflow.org/tutorials).

Note: Notre communauté `TensorFlow` a traduit ces documents. 
Étant donné que les traductions effectuées dans la communauté sont plus optimales, 
rien ne garantit qu'il s'agit d'un reflet exact et à jour de la documentation anglaise officielle. 
Si vous avez des suggestions pour améliorer cette traduction, 
veuillez envoyer une demande d'extraction ou `pull request`) à [tensorflow/docs](https://github.com/tensorflow/docs). 
Pour vous porter volontaire pour écrire ou réviser les traductions de la communauté, 
contactez la [liste docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).